In [2]:
import os

import pandas as pd 
import torch
import torch.nn as nn
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader

In [3]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)

In [4]:
class Data_Reader(Dataset):
    def __init__(self, X:pd.Series, Y:pd.Series, tokenizer, max_len:int = 184) -> None:
        super().__init__()
        self.texts = X['text'].to_list()
        self.labels = Y['label'].to_list()
        self.max_len = max_len
        self.tokenizer = tokenizer
        
    def __len__(self) -> int:
        return len(self.texts)
    
    def __getitem__(self, index) -> list:
        
        encoding = self.tokenizer(self.texts[index], return_tensors='pt',
                                     padding='max_length',max_length = self.max_len,
                                    truncation=True, add_special_tokens=True)
        return [
            torch.tensor(self.labels[index]).long(),
            {'input_ids':encoding['input_ids'],
             'attention_mask':encoding['attention_mask']
            }
            ]

In [68]:
X_train = pd.read_csv("../data/train/X.csv")[:100]
Y_train = pd.read_csv("../data/train/Y.csv")[:100]

X_val = pd.read_csv("../data/val/X.csv")[:20]
Y_val = pd.read_csv("../data/val/Y.csv")[:20]

In [69]:
train_data = Data_Reader(X_train,Y_train,tokenizer,max_len=136)
val_data = Data_Reader(X_val,Y_val,tokenizer,max_len=136)

In [70]:
train_data_batched = DataLoader(train_data, batch_size=8, shuffle=True, drop_last= True)
val_data_batched = DataLoader(val_data, batch_size=8, shuffle=True, drop_last= True)

In [71]:
batch_count = 0
for batch in tqdm(train_data_batched):
    if batch_count == 0:
        labels = batch[0]
        inputs = batch[1]
        for key, value in inputs.items():
            inputs[key] = value.squeeze(1)
        batch_count += 1
    else:
        break

  8%|▊         | 1/12 [00:00<00:00, 33.33it/s]


In [79]:
# class BiLSTM(nn.Module):
#     def __init__(self,hidden_size:int = 256, num_layers:int = 1) -> None:
#         super().__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.bert = BertModel.from_pretrained('bert-base-uncased',output_hidden_states= True)
#         self.drop_bert = nn.Dropout(0.1)
#         self.bilstm = nn.LSTM(input_size=768, hidden_size= hidden_size, num_layers= num_layers, bidirectional= True, batch_first= True)
#         self.dense = nn.Sequential(nn.Linear(in_features= hidden_size*2, out_features= 32),
#                                     nn.ReLU(),
#                                     nn.Linear(in_features= 32, out_features= 6),
#                                     nn.Softmax(dim=1))
        
#         # self.apply(init_weights)
        
#     def forward(self, x):
#         out = self.bert(**x).hidden_states[-2]
#         out = self.drop_bert(out)
#         lstm_out, _ = self.bilstm(out, None)
#         lstm_out = lstm_out[:,-1,:]
#         # for i in range(out.size(-2)):
#         #     if i == 0:
#         #         l_out, (hidden,cell) = self.bilstm(out[:,i,:].unsqueeze(1))
#         #     else:
#         #         l_out, (hidden,cell) = self.bilstm(out[:,i,:].unsqueeze(1),(hidden,cell))
        
#         # l_out = l_out.squeeze(1)
#         final_output = self.dense(lstm_out)
#         return final_output

class BERTClassifier(nn.Module):
    def __init__(self, num_classes= 6):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
        
        # self.apply(init_weights)

    def forward(self, x):
        outputs = self.bert(**x)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [80]:
def validate_accuracy(dataloader, model):
    model.eval()
    acc = 0
    for batch in dataloader:
        labels = batch[0]
        inputs = batch[1]
        for key, value in inputs.items():
            inputs[key] = value.squeeze(1)
        with torch.no_grad():
            output = model(inputs)
        _, preds = torch.max(output, dim=1)

        hits = sum(preds == labels)
        acc += hits/labels.size(0)
    avg_acc = acc/len(dataloader)
    
    return avg_acc

In [77]:
bertclassifier = BERTClassifier()
# bilstm = BiLSTM(hidden_size= 128)
model_opt = torch.optim.Adam(params= bertclassifier.parameters(), lr= 5e-5)
loss_function = nn.CrossEntropyLoss()

In [78]:
batch_loss, batch_count = [], 0

for epoch in range(2):
    # val_acc = validate_accuracy(val_data_batched, bertclassifier).item()
    # print(f"Epoch: {epoch}, Val Acc: {val_acc*100:.2f}%")
    for batch in tqdm(train_data_batched):
        bilstm.train()
        batch_count += 1
        labels = batch[0]
        inputs = batch[1]
        for key, value in inputs.items():
            inputs[key] = value.squeeze(1)
        
        output = bilstm(inputs)
        loss = loss_function(output,labels)
        
        model_opt.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(bilstm.parameters(), 2)
        model_opt.step()
        
        val_acc = validate_accuracy(val_data_batched, bilstm).item()
        
        print(f"Batch: {batch_count} \t Loss: {loss.item():.5f} \t Val Acc: {val_acc*100:.2f}%")
        batch_loss.append(loss.item())
    

  8%|▊         | 1/12 [00:13<02:23, 13.01s/it]

Batch: 1 	 Loss: 1.77956 	 Val Acc: 56.25%


 17%|█▋        | 2/12 [00:25<02:09, 12.90s/it]

Batch: 2 	 Loss: 1.78419 	 Val Acc: 62.50%


 25%|██▌       | 3/12 [00:38<01:54, 12.74s/it]

...

Batch: 22 	 Loss: 1.80556 	 Val Acc: 18.75%


 92%|█████████▏| 11/12 [02:31<00:13, 13.24s/it]

Batch: 23 	 Loss: 1.77968 	 Val Acc: 12.50%


100%|██████████| 12/12 [02:44<00:00, 13.72s/it]

Batch: 24 	 Loss: 1.76044 	 Val Acc: 18.75%


In [25]:
def save_model(model, filepath):
    """
    Save PyTorch model parameters to a file.

    Args:
    - model (torch.nn.Module): PyTorch model to save.
    - filepath (str): Filepath to save the model parameters.
    """
    torch.save(model.state_dict(), filepath)
    print(f"Model parameters saved to '{filepath}'")

def load_model(model, filepath):
    """
    Load PyTorch model parameters from a file.

    Args:
    - model (torch.nn.Module): PyTorch model to load parameters into.
    - filepath (str): Filepath to the saved model parameters.
    """
    model.load_state_dict(torch.load(filepath))
    print(f"Model parameters loaded from '{filepath}'")


In [ ]:
save_model(bertclassifier, "bert_model2.pth")

Model parameters saved to 'bert_model.pth'
